# Start cuda

In [1]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


# Loading requirements

In [2]:
! pip install datasets

     |████████████████████████████████| 163kB 12.7MB/s 
     |████████████████████████████████| 17.7MB 177kB/s 
     |████████████████████████████████| 245kB 56.6MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [3]:
! pip install transformers

     |████████████████████████████████| 1.4MB 12.3MB/s 
     |████████████████████████████████| 2.9MB 52.5MB/s 
     |████████████████████████████████| 890kB 48.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=572e3e52c76026af0ad2a491fe00084fe994f40b1cec9aa6e27cc38f2d71237c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
# Import things to get data to work
from datasets import load_from_disk
import sys
import os

# Loading data

In [5]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
sys.path.append(os.path.join('/content/drive/My Drive/deep_learning_project'))
train_path = 'drive/My Drive/deep_learning_project/all_data/train_all'
val_path = 'drive/My Drive/deep_learning_project/all_data/validation_all'

In [19]:
train_data = load_from_disk(train_path)
validation_data = load_from_disk(val_path)

In [20]:
print(train_data[0])

{'answer': 'york', 'paragraph': 'judi dench born in dorset and later moved to dublin where he was raised he met dench s mother while he was studying medicine at trinity college dublin dench attended the mount school a quaker independent secondary school in york and became a quaker her brothers one of whom was actor jeffery dench were born in tyldesley lancashire her niece emma dench is a roman historian and professor previously at birkbeck university of london and currently at harvard university career in britain dench has developed a reputation as one of the greatest actresses of the post war period primarily through her work in theatre which has been her forte throughout her career she has more than once been named number one in polls for britain s best actor early years', 'question': 'Where in England was Dame Judi Dench born?', 'question_id': 'tc_3'}


# Preparing data for modelling

In [21]:
from math import ceil
import torch
from transformers import AutoTokenizer, BertModel, AdamW, BertForSequenceClassification

In [22]:
def get_loss(sim):
    nll = -(torch.diagonal(sim) - torch.logsumexp(sim, dim = 1))
    return sum(nll)# return negative loss

In [25]:
# Define data parameters
batch_size = 100

# Train
n_sample_train = ceil(len(train_data)*0.70) 
n_batches_train = ceil(n_sample_train/batch_size)
# Validation
n_sample_validation = ceil(len(validation_data)*0.70)
n_batches_validation = ceil(n_sample_validation/batch_size)


# Define model parameters
lr = 5e-5
n_epochs = 4

# Define printing parameters
n_batch_print = 50 # Prints every (n_batch_print) during training

In [26]:
print(n_sample_validation)
print(n_sample_train)

11553
87579


In [27]:
# NB CHANGE THIS CELL WHEN THE DATA IS READY###############
# Subset data
train_data = train_data.select(range(n_sample_train))
validation_data = validation_data.select(range(n_sample_validation))

# Tokenize data
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', padding = True)
train_data = train_data.map(lambda example: {
    'Q_input_ids': tokenizer(example['question'], padding = 'max_length')['input_ids'],
    'Q_attention_mask': tokenizer(example['question'], padding = 'max_length')['attention_mask'],
    'Q_token_type_ids': tokenizer(example['question'], padding = 'max_length')['token_type_ids'],
    'P_input_ids': tokenizer(example['paragraph'], padding = 'max_length')['input_ids'],
    'P_attention_mask': tokenizer(example['paragraph'], padding = 'max_length')['attention_mask'],
    'P_token_type_ids': tokenizer(example['paragraph'], padding = 'max_length')['token_type_ids']},
    batched = True, batch_size= batch_size)

validation_data = validation_data.map(lambda example: {
    'Q_input_ids': tokenizer(example['question'], padding = 'max_length')['input_ids'],
    'Q_attention_mask': tokenizer(example['question'], padding = 'max_length')['attention_mask'],
    'Q_token_type_ids': tokenizer(example['question'], padding = 'max_length')['token_type_ids'],
    'P_input_ids': tokenizer(example['paragraph'], padding = 'max_length')['input_ids'],
    'P_attention_mask': tokenizer(example['paragraph'], padding = 'max_length')['attention_mask'],
    'P_token_type_ids': tokenizer(example['paragraph'], padding = 'max_length')['token_type_ids']},
    batched = True, batch_size= batch_size)

#%% Change to pytorch format. 
train_data.set_format(type = 'torch', 
                        columns = ['Q_input_ids', 'Q_attention_mask', 'Q_token_type_ids',
                                   'P_input_ids', 'P_attention_mask', 'P_token_type_ids'])

validation_data.set_format(type = 'torch', 
                        columns = ['Q_input_ids', 'Q_attention_mask', 'Q_token_type_ids',
                                   'P_input_ids', 'P_attention_mask', 'P_token_type_ids'])

Loading cached processed dataset at drive/My Drive/deep_learning_project/all_data/train_all/cache-17b2039bdcf4806b.arrow


# Training model

In [28]:
# Get pretrained model
model = BertModel.from_pretrained('bert-base-uncased')
#model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

#Freezing all parameters except the last ones (I think - double check this)
param_to_be_optimized = ['pooler.dense.weight', 'pooler.dense.bias']
for name, param in model.named_parameters():
  #print(name)
  if not name in param_to_be_optimized:
    param.requires_grad = False

# --------- From Hugging Face ---------------
#for name, param in model.base_model.named_parameters():
#  param.requires_grad = False
  #if(param.requires_grad): 
   # print(name)
#for name, param in model.named_parameters():
#  if(param.requires_grad): 
#    print(name)
# -----------------------------------------

# Checking we have frozen the right layers 
for name, param in model.named_parameters():
  if(param.requires_grad): 
    print(name)

# Move model to cuda to train there
model.to(device)

optim = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr = lr) # filter object works as a generator 


pooler.dense.weight
pooler.dense.bias


In [29]:
# ADAM gets the correct bias and weigths
optim.param_groups[0]['params'][1].size()

torch.Size([768])

In [30]:
# Printing layers in model to compare 
#model

In [31]:
# Print the weights that are being updated
for name, param in model.named_parameters():
  if param.requires_grad:
    print(name)

pooler.dense.weight
pooler.dense.bias


In [32]:
# The big loop :D 
epoch_train_loss = [None]*n_batches_train
epoch_validation_loss = [None]*n_batches_validation
train_loss = [None]*n_epochs
validation_loss = [None]*n_epochs


for epoch in range(n_epochs):
    
    print(f'### EPOCH: {epoch+1}/{n_epochs} ###')

    trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
    trainloader = iter(trainloader)
    validationloader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size)
    validationloader = iter(validationloader)
    
    # TRAINING MODEL
    model.train()
    for i, batch in enumerate(trainloader):
        if i % n_batch_print == 0:
          print(f'batch {i+1}/{len(trainloader)}')
        
        model.zero_grad()

        # (get extra observation in training set)
        # Make forward pass for paragraphs
        input_ids = batch['P_input_ids'].to(device)
        attention_mask = batch['P_attention_mask'].to(device)
        token_type_ids = batch['P_token_type_ids'].to(device)
        P_encoded_layers = model(input_ids=input_ids, 
                          attention_mask=attention_mask, 
                          token_type_ids=token_type_ids)[0][:, 0, :]

        # Make forward pass for questions
        input_ids = batch['Q_input_ids'].to(device)
        attention_mask = batch['Q_attention_mask'].to(device)
        token_type_ids = batch['Q_token_type_ids'].to(device)
        Q_encoded_layers = model(input_ids=input_ids, 
                          attention_mask=attention_mask, 
                          token_type_ids=token_type_ids)[0][:, 0, :]
    
    
        # Calculate similarity matrix
        sim_matrix = torch.matmul(Q_encoded_layers, P_encoded_layers.T)
        
        # Get loss
        loss = get_loss(sim_matrix)
        loss.requires_grad = True

        # Update weights
        loss.backward()
        optim.step()

        # Save loss
        epoch_train_loss[i] = loss.item()

    # VALIDATING MODEL
    #model.eval()
    for i, batch in enumerate(validationloader):
        
        # (get extra observation in training set)

        # Make forward pass for paragraphs
        input_ids = batch['P_input_ids'].to(device)
        attention_mask = batch['P_attention_mask'].to(device)
        token_type_ids = batch['P_token_type_ids'].to(device)
        P_encoded_layers = model(input_ids=input_ids, 
                          attention_mask=attention_mask, 
                          token_type_ids=token_type_ids)[0][:, 0, :]

        # Make forward pass for questions
        input_ids = batch['Q_input_ids'].to(device)
        attention_mask = batch['Q_attention_mask'].to(device)
        token_type_ids = batch['Q_token_type_ids'].to(device)
        Q_encoded_layers = model(input_ids=input_ids, 
                          attention_mask=attention_mask, 
                          token_type_ids=token_type_ids)[0][:, 0, :]
    
    
        # Calculate similarity matrix
        sim_matrix = torch.matmul(Q_encoded_layers, P_encoded_layers.T)
        
        # Get loss
        loss = get_loss(sim_matrix)
        epoch_validation_loss[i] = loss.item()
        
    
    train_loss[epoch] = sum(epoch_train_loss)/len(epoch_train_loss)
    validation_loss[epoch] = sum(epoch_validation_loss)/len(epoch_validation_loss)
    
    print(f'train loss: {train_loss[epoch]:.2f}')
    print(f'validation loss: {validation_loss[epoch]:.2f}')
    




### EPOCH: 1/4 ###


/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py:850: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


batch 1/876
batch 51/876
batch 101/876
batch 151/876
batch 201/876
batch 251/876
batch 301/876


KeyboardInterrupt: ignored

In [ ]:
#model

In [ ]:
print(validation_loss)
print(training_loss)

In [ ]:
#%% Saving model
model_path = 'drive/My Drive/deep_learning_project/model_bert_base_only_outer_layers/'
model.save_pretrained(model_path)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(range(1, n_epochs+1), train_loss, label = 'Train loss')
plt.plot(range(1, n_epochs+1), validation_loss, label = 'Validation loss')
plt.title('Finetuning BERT Base Model')
plt.ylabel('Loss')
plt.legend()
plt.xlabel('Epoch')
plt.show()
fig.savefig("drive/My Drive/deep_learning_project/figures/BERTbaseFinetuning.pdf", bbox_inches='tight')

In [ ]:
validation_loss
train_loss

In [ ]:
print(validation_loss)

In [ ]:
print(train_loss)